In [2]:
# importing libraries
from bs4 import BeautifulSoup
import requests
import nltk; from nltk.corpus import stopwords; from nltk.stem import PorterStemmer
import heapq
import pandas as pd
import math
from sklearn.metrics.pairwise import cosine_similarity

#==========================================================================================================================
# Part related to the crawling of wikipedia
#==========================================================================================================================

# doing the request to the web page with the films
film_list = []; app = []
for i in range(3):
    request = requests.get("https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies" + str(i+1) 
                           + ".html")
    soup = BeautifulSoup(request.text, 'html.parser')

    # extract all the links that are into the page and insert all the links into a list
    app = soup.select("a")
    for i in range(len(app)):
        film_list.append(app[i].get("href"))
    

# for each link that is into the list created previously extract the HTML code related to that link and insert it into a
# file created at the moment into the position specified by the url_file variable
for i in range(len(film_list)):
    url_file = "C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles3\\article_" +str(i+1)+ ".html"
    file = open(url_file, "w", encoding="utf-8")
    file.write(str(requests.get(film_list[i]).text))
    file.close()

#==========================================================================================================================
# That part extract all needed informations  from the crawled pages
#==========================================================================================================================

# extracting data from crawled HTML pages and transform it into a film.tsv file with the extracted informations as specified
# into the requests
for i in range(len(film_list)):
    # inizializing variables
    flag = 0; 
    director = "NA"; producer = "NA"; writer = "NA"; starring = "NA"; music = "NA"; release_date = "NA"; runtime = "NA"; 
    country = "NA"; language = "NA"; budget = "NA"
    
    # creating url, open file and parsing it
    url = "C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles\\article_" + str(i+1) + ".html"
    fileIn = open(url, "r", encoding="utf-8")
    fileOut = open("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles\\film" + str(i+1) + ".tsv", "a", encoding="utf-8")
    film = BeautifulSoup(fileIn.read(), "html.parser")
    
    # extract title of the page and the film
    list_h1 = film.select("h1")
    for h1 in list_h1:
        if h1.get("id") == "firstHeading":
            fTitle = h1.text
            
    # this part trys to extract the intro and plot of the film, if they aren't into the page they are set to NA
    try:
        intro = film.select("p")[0].text
        plot = film.select("p")[1].text
    except:
        intro = "NA"; plot = "NA"
    finally:
    
        # search and then extract the required values from the infobox table related to the film
        fTables = film.select("table")
        for table in fTables:
            if table.get("class") != None:
                if "infobox" in table.get("class"):
                    rows = table.select("tr")
                    for row in rows:
                        for el in row.select("th"):
                            if "class" in el.attrs.keys():
                                if "summary" in el.attrs["class"]:
                                    flag = 1
                                    summary = el.text
                                else:
                                    summary = "NA"
                            if el.text == "Directed by":
                                director = row.select("td")[pos].text
                            if el.text == "Produced by":
                                producer = row.select("td")[pos].text
                            if el.text == "Written by":
                                writer = row.select("td")[pos].text
                            if el.text == "Starring":
                                starring = row.select("td")[pos].text
                            if el.text == "Music by":
                                music = row.select("td")[pos].text
                            if el.text == "Release date":
                                release_date = row.select("td")[pos].text
                            if el.text == "Running time":
                                runtime = row.select("td")[pos].text
                            if el.text == "Country":
                                country = row.select("td")[pos].text
                            if el.text == "Language":
                                language = row.select("td")[pos].text
                            if el.text == "Budget":
                                budget = row.select("td")[pos].text
                            pos += 1
                        pos = 0

        # write into the result file all the important data extracted according to the format of a tsv file
        if flag == 0: summary = "NA"
        fileOut.write(fTitle + " \t " + intro + " \t " + plot + " \t " + summary + " \t " + director + " \t " + producer + " \t " + 
                      writer + " \t " + starring + " \t " + music + " \t " + release_date + " \t " + runtime + " \t " + country +
                      " \t " + language + " \t " + budget)
    
    # close the files that aren't used
    fileIn.close(); fileOut.close()

#==========================================================================================================================
# This park makes the indexing of the extracted informations 
#==========================================================================================================================

# declarating global variables
rev_index_App = {}; rev_index = {}
commonWords = set(stopwords.words("english"))

# does a first reverse indexing, at the end the result is represented by a dictionary with words as a key and the value
# associated is a list of integers where each integer represent the number of the page where the word compares. Ex:
# {"word_1": [1, 2, 4, 5], "word_2": [2, 5, 6] ...}
# so the word_a is into the first, second, fourth and fifth pages, the word_2 is into second, fifth and sixth page and so on
for i in range(1, 10000):
    # opening files and variables
    fileIn = open("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles\\film" + str(i) + ".tsv", encoding="utf-8")
    intro_plot = []
    fileText = fileIn.read()
    
    # extract the summary, intro and the plot of each film article
    for j in range(3):
        intro_plot.append(fileText[:fileText.find("\t")].strip())
        fileText = fileText[fileText.find("\t") + 2:]

    # first removes the summary because it isn't required for the indexing of the words
    # then does a parsing of the string represented by the el variable transforming it into a list of words then
    # controls if the word is "legit" and insert it into a list that will be used after to construct a first result
    # dictionary as exposed into the comment that begins on the row 10
    intro_plot.pop(0)
    for el in intro_plot:
        listaParole = nltk.word_tokenize(el); listaParoleAcc = []
        for j in range(len(listaParole)):
            if listaParole[j].isalpha() == True and listaParole[j].lower() not in commonWords:
                listaParoleAcc.append(listaParole[j].lower())
        for word in listaParoleAcc:
            if word not in rev_index_App.keys():
                rev_index_App[word] = [i]
            else:
                if i not in rev_index_App[word]:
                    rev_index_App[word].append(i)
    fileIn.close()

# this part creates a matrix with 2 columns and 10000 rows, each row contains the id of the word and the word associated
# Ex: [[1, word_1], [2, word_2], [3, word_3] ...]
dict_id = []
for i in range(len(rev_index_App.keys())):
    dict_id.append([i+1, list(rev_index_App.keys())[i]])

# this part creates a new dictionary where the keys aren't the words but their index
# Ex: {1: [1, 2, 3, 4], 2: [2, 4, 6, 7] ...}
value = list(rev_index_App.values())
for i in range(len(rev_index_App.keys())):
    rev_index[i+1] = value[i]

# writes into two different files the matrix with the "conversion table" from an id to his word and the reverse index
# dictionary
fileIds = open("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles\\vocabulary.txt", "w", encoding="utf-8")
fileIds.write(str(dict_id))
fileIds.close()
fileDict = open("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles\\reverseIndex.txt", "w", encoding="utf-8")
fileDict.write(str(rev_index))
fileDict.close()

#==========================================================================================================================
# This part makes a film research using cosine similarity from each document 
#==========================================================================================================================

#to execute the query in an easiest way
dict_id_1 = {}
for i in range(len(dict_id)):
    dict_id_1[dict_id[i][1]] = i+1 #form nested list to dict
    
num_query = []

query = input('Insert your search: ').lower()  # so we work with every word in the same format
query = nltk.word_tokenize(query)              # list of tokenized words of the query 

for el in query:
    if (PorterStemmer().stem(el)) in dict_id_1:               # stemmed query
        num_query.append(dict_id_1[PorterStemmer().stem(el)]) # list with query entry converted in the same number given to 
                                                              # the words in my database

query_dictionary = {}   # use this dict to select only words and related docs that i want to check with te query
nested_list = []

for el in num_query:
    if el in rev_index.keys():
        query_dictionary[el] = rev_index[el] # a new dict with only the key value I need
        nested_list.append(rev_index[el])

# Now we procced doing the set of sublist that have the documents in wich is contained each of the query's word.
# So the intersection between them, is a list in wich there are only the documents that have all the words requested!

s = set(i for sub in nested_list for i in sub) # unique list of all documents that have all query
query_result = [i for i in s if all(i in sub for sub in nested_list)] # find all docs that have al words of query doing the
                                                                      # intersection of sets
# Now we have to create a unique list of film links, because before we worked in parallel

query_df = pd.DataFrame(); l = []

for i in sorted(query_result): # for film in series, respect chronological order
    url = "C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles\\film" + str(i) + ".tsv"
    fileIn = open(url,'r',encoding='utf-8')
    s = fileIn.read()
    s1 = s.replace('\t \n \t','\t') #'\n' gives a problem about splitting the text 
    s_splitted = s1.split('\t')

    l.append([s_splitted[0], s_splitted[1], film_list[i-1]])
    
query_df = pd.DataFrame(l)
query_df.columns=['Title','Intro','Wikipedia link']

# Now for this query we need to do some changes on the previous query code

dict_id = []
dictionary_id = {} 
for i in range(len(rev_index_App.keys())):
    a = list(rev_index_App.keys())[i]
    dictionary_id[a]= i+1
    
word_id_list = [] # each sub has the words of a document, stored by their Id

for sub in range(len(words_doc_i)): # each sublist
    l_temp=[]
    for word in range(len(words_doc_i[sub])): # each word
        if words_doc_i[sub][word] in dictionary_id.keys():
            l_temp.append(dictionary_id[words_doc_i[sub][word]])
    word_id_list.append(l_temp)
    
tfDict_final = {}
for i in range(docs):
    tfDict = {} #need one per document
    words =  word_id_list[i]
    words_number = len(word_id_list[i])    #counting all the words inisde the doc
    
    numOfWords_i = dict.fromkeys(set(words), 0) #every word in the document with start occurency 0
    for word in words:
        numOfWords_i[word] += 1 #dict with words of document(i) and their occurences
    
    for word in numOfWords_i:
        try:
            tfDict[word] = round(numOfWords_i[word] / float(words_number), 4) #appplication of the rule to calculate 'tf'
        except:
            continue
    
    tfDict_final[i]=tfDict

value = list(rev_index_App.values())
for i in range(len(rev_index_App.keys())):
    rev_index[i+1] = value[i]               # dict with each word and in which docs we can find it 
    
all_set_occ = {} # the frequency of each word over all the doc set

for key in rev_index:
    summary = len(rev_index[key]) # rev_index[key] is a list of all docs where is a certain word, so we need the lenght of this 
                                  # list
    newkey = int(key)
    all_set_occ[newkey] = round(math.log(docs/summary), 4) 

# Now we'll build a new dictionary, in wich for each key (word) we can see in wich docs is contained and also for each one 
# of the the tf-idf score.

word_list = list(dictionary_id.values())
 
Rev_Index_1 = {}
    
for word in word_list:
    l_doc = rev_index[word]   #list of all documents with that word
    for doc in l_doc:
        words_tf = tfDict_final[doc]         #all the words in Doc(i) with their tf
        for doc_word in words_tf.keys():     #for each word of each doc
            Idf = all_set_occ[doc_word]           #find the idf of that word
            TfIdf = round((words_tf[doc_word] * Idf),3) #calculate the tf-idf score
            
            if doc_word not in Rev_Index_1.keys(): #if the word is not in the dict add it and add the first doc with and the tfidf
                Rev_Index_1[doc_word] = [(doc,TfIdf)]
            else:
                if (doc,TfIdf) not in Rev_Index_1[doc_word]: #if the word is already in the dict we update the values
                    Rev_Index_1[doc_word].append((doc,TfIdf))

queryIn = input('Insert your search :').lower()
elems_query = nltk.word_tokenize(queryIn)  #list of words of the query
numerical_query = []


for el in elems_query:
    if (PorterStemmer().stem(el)) in dict_id_1:               #stemmed query
        numerical_query.append(dict_id_1[PorterStemmer().stem(el)]) #list with query entry converted in the same number given to the words in my database

query_dict = {}   #use this dict to select only rows and columns I want to checj
list_nested = []
l_tfidf = []
dict_word_l_tfidf = {}

for el in numerical_query:
    if el in Rev_Index_1.keys():
        l_tfidf_temp = []
        query_dict[el] = Rev_Index_1[el]   # a dict with only the values we need to execute the query
        list_nested.append(Rev_Index_1[el]) # a list with only the values we need to execute the query
        
        docs_list = query_dict[el]
        
        for value in docs_list:
            l_tfidf_temp.append(value[1]) 
    l_tfidf.append(l_tfidf_temp)
    
    dict_word_l_tfidf[el] = l_tfidf_temp  #new dict, keys are words and values the list of their tfidf
    
set_list = []
docs_nested_list = []

for sub in list_nested: #sub is formed by a list of documents
    l = []
    for elem in sub:
        set_list.append(elem[0])
        l.append(elem[0])
    
    docs_nested_list.append(l)

set_list = set(set_list)

query_results = [i for i in set_list if all(i in sub for sub in docs_nested_list)] # find all docs that have al words of 
                                                                                   # query doing the intersection of sets
    
query_dict_tfidf = {}
for key in query_dict:
    num_of_docs = dict.fromkeys(total_docs, 0)
    for doc_tfidf in query_dict[key]:
        num_of_docs[doc_tfidf[0]] += doc_tfidf[1]
    
    doc_list = list(num_of_docs.values())
    query_dict_tfidf[key] = doc_list    

# this dic has every word of the query as a key, and as a values a list of tfidf for each document, from the first to the last
# of the dataset, so we have a value if the word is inside a doc or 0 if there isn't. This format is the one we need to compute
# cos similarity between query results

query_df = pd.DataFrame.from_dict(query_dict_tfidf ) #by index are not ordered 

query_results_sort = sorted(query_results)
query_norm = []
for i in query_results_sort:
    new = i-1
    query_norm.append(new)
    
newdf_query = query_df.loc[query_norm]

query_for_cosine = []
for i in numerical_query:
    query_for_cosine.append(1) #format needed to perform the cos sim between the query and the dataframe's rows 


cosine_results = []
cosine_results = cosine_similarity([query_for_cosine], newdf_query)
cosine_results = cosine_results.tolist()

for elem in cosine_results:
    el = elem
cosine_results_1 = el

l_title = []; l_intro = []; l_link = []
for i in (query_results_sort): #for film in series, respect chronological order
    url = 'C:\\Users\\Emanuele Fratocchi\\Desktop\\Data Science\\Articles\\tsv_HW3\\film' + str(i) +'.tsv'
    fileIn = open(url,'r',encoding='utf-8')
    s = fileIn.read()
    s1 = s.replace('\t \n \t','\t') #'\n' gives a problem about splitting the text 
    s_splitted = s1.split('\t')

    l_title.append(s_splitted[0]) #building the columns of the dataframe
    l_intro.append(s_splitted[1])
    l_link.append(film_list_final[i-1])

query_resDic = {}
for key_index in range(len(query_results_sort)):
    query_resDic[query_results_sort[key_index]] = [l_title[key_index], l_intro[key_index], l_link[key_index],cosine_results_1[key_index]]

Res_df = pd.DataFrame(query_resDic)
Res_df = Res_df.transpose()    
Res_df.columns = ['Title', 'Intro','Wikipedia Link','Cos Similarity']
Res_df = Res_df.sort_values('Cos Similarity',ascending = False)
Res_df.reset_index(inplace=True)
Res_df = Res_df[['Title', 'Intro','Wikipedia Link','Cos Similarity']]

if len(query_results_sort) <= 10:
    Res_df = Res_df.head()
elif len(query_results_sort) > 10:
    Res_df = Res_df.head(10)

print(Res_df)

#==========================================================================================================================
# This part makes a film research using weighted scores 
#==========================================================================================================================

# this function controls if at least one word of the query matches with the words contained into an element of the film, if 
# this happen returnd a positive score, otherwise it returns 0
def assPunt(lisParAcc, lisParQuery, i):
    for q in lisParQuery:
        if q.lower() in lisParAcc:
            if 1 <= i <= 3:
                return 19
            else:
                return 4
    return 0

# creation of global variables
firstRes = {}; res = []; commonWords = set(stopwords.words("english"))

# handles the query and makes the reseach with the score
query = input("Inserisci le parole chiave della tua ricerca: ")
query = query.split(" ")
for i in range(10000):
    fileIn = open("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles1\\film" + str(i+1) + ".tsv", encoding="utf-8")
    info = []
    fileText = fileIn.read()
    tot = 0

    while fileText.find("\t") != -1:
        info.append(fileText[:fileText.find("\t")].strip())
        fileText = fileText[fileText.find("\t") + 2:]
    info.append(fileText.strip())

    # gives a score to the document and creates a dictionary with the number of the documents with highter score
    for el in info:
        cont = 1
        listaParole = nltk.word_tokenize(el); listaParoleAcc = []
        for word in listaParole:
            word = word.lower()
            if word.isalpha() == True and word not in commonWords and PorterStemmer().stem(word) not in listaParoleAcc:
                listaParoleAcc.append(PorterStemmer().stem(word))
        tot += assPunt(listaParoleAcc, query, cont)
        cont += 1

    if len(firstRes) < 10:
        firstRes[i+1] = round(tot/101, 4)
    else:
        if round(tot/101, 4) > min(firstRes.values()):
            for key in firstRes.keys():
                if firstRes[key] == min(firstRes.values()):
                    remKey = key
            firstRes.pop(remKey)
            firstRes[i+1] = round(tot/101, 4)

# creates the heap tree and extract the values from the tree (in practical it makes an heapSort)
heap = list(firstRes.values())
heapq.heapify(heap)
for i in range(len(firstRes)):
    res.insert(0, heapq.heappop(heap))

# search the number of the document with a certain scores and then print the required elements(title, intro, link, and score)
for el in res:
    for key, value in firstRes.items():
        if value == el:
            fileIn = open("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles1\\film" + str(key) + ".tsv", encoding="utf-8")
            fileText = fileIn.read()
            s = ""
            for j in range(2):
                s += fileText[:fileText.find("\t")].strip() + "\t"
                fileText = fileText[fileText.find("\t") + 2:]
            s += film_list[key-1] + "\t" + str(value)
            print(s)